In [1]:
#loading the dataset
from datasets import load_dataset
dataset = load_dataset('dair-ai/emotion')

In [2]:
#inspecting dataset
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
{'text': 'i didnt feel humiliated', 'label': 0}


In [3]:
#preprocessing and tokenization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [4]:
#tokenization
def tokenize_function(example):
    return tokenizer(example['text'], padding = 'max_length', truncation = True, max_length = 128)

tokenized_dataset = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
#loading the model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6  # Match the number of emotion classes
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
dataset["train"].features["label"].names

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [16]:
#training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./emotion-distilbert",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)


In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [18]:
#the trainer
from transformers import Trainer, TrainingArguments, TrainerCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6944\1829483507.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#start finetuing/training a pretrained model
trainer.train()

Step,Training Loss


In [13]:
dataset["train"].features["label"]


ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)

In [ ]:
#Saving model
model.save_pretrained("./emotion-distilbert-model")

In [ ]:
#Saving tokenizer
tokenizer.save_pretrained("./emotion-distilbert-model")